In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, Dates, Plots, TimeSeries, JSON, Distributions, ExpectationMaximization, StatsBase

In [2]:
data_directory = joinpath("/projects", "emco4286", "data", "gads", "trajectories", "st", "long")
files = glob("gen_*_class_ST_rating_*_state_Texas.csv", data_directory)

133-element Vector{String}:
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_3_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 30 bytes ⋯ "ass_ST_rating_3_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_3_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_ST_rating_2_state_Texas.csv"
 ⋮
 "/projects/emco4286/data/gads/tr" ⋯ 30 bytes ⋯ "ass_ST_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 30 bytes ⋯ "ass_ST_rating_2_st

In [3]:
ph = 8760

foh = 380.401
num_fo = 5.679

poh = 234.932
num_po = 1.859

moh = 67.509
num_mo = 0.74

fo_ttf_rate = (ph - moh - poh)/num_fo
po_ttf_rate = (ph - moh - foh)/num_po
mo_ttf_rate = (ph - foh - poh)/num_mo

@printf "TTF FO: %0.2f hours\n" fo_ttf_rate
@printf "TTF PO: %0.2f hours\n" po_ttf_rate
@printf "TTF MO: %0.2f hours\n" mo_ttf_rate

TTF FO: 1489.27 hours
TTF PO: 4471.27 hours
TTF MO: 11006.31 hours


In [4]:
ttf_values_fo_vec = Any[]
ttr_values_fo_vec = Any[]

ttf_values_po_vec = Any[]
ttr_values_po_vec = Any[]

ttf_values_mo_vec = Any[]
ttr_values_mo_vec = Any[]

outage_types_vec = []

for f in files

    df = DataFrame(CSV.File(f))

    if size(df)[1] < 1
        continue
    end

    outage_start = findall(diff(df[!, :y1]) .== -1)
    outage_end = findall(diff(df[!, :y1]) .== 1)

    # If the plant data starts with an outage, don't count the end of the first outage because we won't know the actual TTR
    if df[1, :y1] == 0
        outage_end = outage_end[begin + 1: end]
    end

    if length(outage_end) < length(outage_start)
        outage_start = outage_start[begin: end-1]
    elseif length(outage_end) > length(outage_start)
        outage_end = outage_end[begin + 1: end]
    end

    ttr_values = outage_end .- outage_start

    if length(filter(x -> x < 0, ttr_values)) > 0
        print(f)
        break
    end
    
    ttf_values_all_types = outage_start[begin + 1: end] - outage_end[begin:end - 1]

    outage_types = df[outage_start .+ 1, :y2]
    push!(outage_types_vec, outage_types)
    
    fo_idx = findall(outage_types .== 2)
    os_fo = outage_start[fo_idx]
    oe_fo = outage_end[fo_idx]

    fo_idx_mod = filter(x -> x > 0, fo_idx .- 1)

    ttf_values_fo = os_fo[begin + 1: end] - oe_fo[begin:end - 1] # ttf_values_all_types[fo_idx_mod]
    ttr_values_fo = ttr_values[fo_idx]

    push!(ttf_values_fo_vec, ttf_values_fo)
    push!(ttr_values_fo_vec, ttr_values_fo)

    po_idx = findall(outage_types .== 3)
    os_po = outage_start[po_idx]
    oe_po = outage_end[po_idx]

    po_idx_mod = filter(x -> x > 0, po_idx .- 1)

    ttf_values_po = os_po[begin + 1: end] - oe_po[begin:end - 1] # ttf_values_all_types[po_idx_mod]
    ttr_values_po = ttr_values[po_idx]

    push!(ttf_values_po_vec, ttf_values_po)
    push!(ttr_values_po_vec, ttr_values_po)

    mo_idx = findall(outage_types .== 4)
    os_mo = outage_start[mo_idx]
    oe_mo = outage_end[mo_idx]

    mo_idx_mod = filter(x -> x > 0, mo_idx .- 1)

    ttf_values_mo = os_mo[begin + 1: end] - oe_mo[begin:end - 1] # ttf_values_all_types[mo_idx_mod]
    # ttf_values_mo = os_mo[begin + 1: end] - oe_mo[begin:end - 1]
    ttr_values_mo = ttr_values[mo_idx]

    push!(ttf_values_mo_vec, ttf_values_mo)
    push!(ttr_values_mo_vec, ttr_values_mo)

end

In [11]:
a = reduce(vcat, outage_types_vec)
total = sum([count(==(element),a) for element in unique(a) ])
my_dict = Dict([element => count(==(element),a) for element in unique(a)])

fo_proportion = (my_dict[2.0])/total
po_proportion = my_dict[3.0]/total
mo_proportion = my_dict[4.0]/total

ttr_values_fo_vec = reduce(vcat, ttr_values_fo_vec)
ttr_values_po_vec = reduce(vcat, ttr_values_po_vec)
ttr_values_mo_vec = reduce(vcat, ttr_values_mo_vec)

ttr_f = fit_mle(Exponential(1/foh), ttr_values_fo_vec)
ttr_p = fit_mle(Exponential(1/poh), ttr_values_po_vec)
ttr_m = fit_mle(Exponential(1/moh), ttr_values_mo_vec)

print("$(round(ttr_f.θ, digits=2)), $(round(ttr_p.θ, digits=2)), $(round(ttr_m.θ, digits=2))")

101.94, 414.01, 71.94

In [12]:
ttf_values_fo_vec = reduce(vcat, ttf_values_fo_vec)
ttf_values_po_vec = reduce(vcat, ttf_values_po_vec)
ttf_values_mo_vec = reduce(vcat, ttf_values_mo_vec)

ttf_f = fit_mle(Exponential(1/fo_ttf_rate), ttf_values_fo_vec)
ttf_p = fit_mle(Exponential(1/po_ttf_rate), ttf_values_po_vec)
ttf_m = fit_mle(Exponential(1/mo_ttf_rate), ttf_values_mo_vec)

print("$(round(ttf_f.θ, digits=2)), $(round(ttf_p.θ, digits=2)), $(round(ttf_m.θ, digits=2))")

361.35, 3531.88, 1931.39

In [13]:
all_ttf = reduce(vcat, [ttf_values_fo_vec, ttf_values_po_vec, ttf_values_mo_vec])
all_ttr = reduce(vcat, [ttr_values_fo_vec, ttr_values_po_vec, ttr_values_mo_vec]);

In [18]:
round.([fo_proportion, mo_proportion, po_proportion], digits=3)

3-element Vector{Float64}:
 0.77
 0.151
 0.079

In [14]:
mix_ttf_guess = MixtureModel([ttf_f, ttf_m, ttf_p], [fo_proportion, mo_proportion, po_proportion])
mix_ttf_mle, _ = fit_mle(mix_ttf_guess, all_ttf, infos = true)

(MixtureModel{Exponential{Float64}}(K = 3)
components[1] (prior = 0.4363): Exponential{Float64}(θ=17.19664372068341)
components[2] (prior = 0.3662): Exponential{Float64}(θ=278.52501060189593)
components[3] (prior = 0.1975): Exponential{Float64}(θ=3666.477697636383)
, Dict{String, Any}("iterations" => 68, "converged" => true, "logtots" => [-188929.39559468126, -186553.03751412153, -185112.0556106915, -184090.2991647176, -183344.14877544373, -182791.31115635694, -182381.00665383087, -182072.03598826725, -181830.11157041718, -181631.2398353454  …  -180545.54160211564, -180545.53609944117, -180545.53166807728, -180545.52809931143, -180545.52522513113, -180545.5229102764, -180545.52104584748, -180545.51954416174, -180545.5183346161, -180545.5173603571]))

In [16]:
mybins = 0:10:maximum(all_ttf)
histogram(all_ttf, bins=mybins, linewidth=0.05, normalize=:pdf, xlims=(0, 2000), labels="Original data", xlabel="Time to failure [hours]", ylabel="Probability density function", title="ST plant outages")
plot!(mybins, pdf(mix_ttf_guess, mybins), label="Guess fit", linewidth=3)
plot!(mybins, pdf(mix_ttf_mle, mybins), label="MLE fit", linewidth=3)
savefig("/home/emco4286/gads-analysis/visualization/ttf_st_mixed_exp_fit_guess_vs_mle.png")

"/home/emco4286/gads-analysis/visualization/ttf_st_mixed_exp_fit_guess_vs_mle.png"

In [19]:
mix_ttr_guess = MixtureModel([ttr_f, ttr_m, ttr_p], [fo_proportion, mo_proportion, po_proportion])
mix_ttr_mle, _ = fit_mle(mix_ttr_guess, all_ttr, infos = true)

(MixtureModel{Exponential{Float64}}(K = 3)
components[1] (prior = 0.2397): Exponential{Float64}(θ=93.94496884336311)
components[2] (prior = 0.6886): Exponential{Float64}(θ=10.200480238868327)
components[3] (prior = 0.0717): Exponential{Float64}(θ=1289.6976015064558)
, Dict{String, Any}("iterations" => 76, "converged" => true, "logtots" => [-134791.83239129005, -132071.88834817347, -131085.46899467634, -130702.60494136397, -130553.34730444002, -130475.57431693765, -130415.7733676716, -130358.89347140436, -130300.84821683171, -130240.55963119317  …  -129260.11347369009, -129260.10885818175, -129260.10511142598, -129260.10206981272, -129260.09960056769, -129260.09759593464, -129260.0959684567, -129260.09464714909, -129260.09357439406, -129260.09270342193]))

In [21]:
mybins = 0:5:maximum(all_ttr)
histogram(all_ttr, bins=mybins, linewidth=0.05, normalize=:pdf, labels="Original data", xlims=(0, 500), xlabel="Time to repair [hours]", ylabel="Probability density function", title="ST plant outages")
plot!(mybins, pdf(mix_ttr_guess, mybins), label="Guess fit", linewidth=3)
plot!(mybins, pdf(mix_ttr_mle, mybins), label="MLE fit", linewidth=3)
savefig("/home/emco4286/gads-analysis/visualization/ttr_st_mixed_exp_fit_guess_vs_mle.png")

"/home/emco4286/gads-analysis/visualization/ttr_st_mixed_exp_fit_guess_vs_mle.png"

In [17]:
ttr_dict = Dict(fieldnames(MixtureModel) .=> getfield.(Ref(mix_ttr_mle), fieldnames(MixtureModel)))
ttf_dict = Dict(fieldnames(MixtureModel) .=> getfield.(Ref(mix_ttf_mle), fieldnames(MixtureModel)))
ct_dict = Dict("ttf" => ttf_dict, "ttr" => ttr_dict)

open("/home/emco4286/gads-analysis/exp_models/st.json","w") do f 
    JSON.print(f, ct_dict) 
end
